<a href="https://colab.research.google.com/github/FANJIYU0825/data-analyst--pratice/blob/fetch_data/finish_varible_name%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#有問題就去看官方文件 很好玩的
簡介:https://research.google.com/colaboratory/faq.html<br>

說明文件:https://colab.research.google.com/notebooks/welcome.ipynb


#爬取大陸國家假統計肺炎網站(扣除台灣/香港/澳門)

##爬蟲

In [0]:
#爬取大陸國家假統計肺炎網站(扣除台灣/香港/澳門)
import requests as res
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
from tqdm import tqdm 
url = 'http://ncov.dxy.cn/ncovh5/view/pneumonia'
response = res.get(url) # text 屬性就是 html 檔案
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "lxml") # 指定 lxml 作為解析器
get_area = soup.find(id="getAreaStat")
area_text = get_area.text
area_catreplace = area_text.replace('}catch(e){}','') #replace 多餘的文字
area_json = area_catreplace.replace('try { window.getAreaStat = ','')#replace 多於文字

##area_json 是一個字串(str) 把字串轉化(list) 

In [0]:
import json
area_json = json.loads(area_json)
provinceName = []
for area_select in area_json:
    if area_select['provinceName'] !='香港':
      if area_select['provinceName'] !='台湾':
        if area_select['provinceName'] !='澳门':
          provinceName.append(area_select)
          

當日以及 所天數檔案的`json`{statisticsData}	

In [3]:
today_df = pd.DataFrame(provinceName)
today_df.head(1)

,provinceName,provinceShortName,currentConfirmedCount,confirmedCount,suspectedCount,curedCount,deadCount,comment,locationId,statisticsData,cities
0,内蒙古自治区,内蒙古,25,227,34,201,1,,150000,https://file1.dxycdn.com/2020/0223/783/3398299...,"[{'cityName': '境外输入人员', 'currentConfirmedCount..."


##right_join dataFrame

{`provinceName`} {`statisticsData`}

In [0]:
all_data_url = today_df.loc[:,['provinceName','statisticsData']]

In [6]:
# 利用2輪迴圈取數值
to_names =[]
to_prevname = []
with tqdm(total = len(all_data_url) ) as pbar:
  for x in range (len(all_data_url)) :
    df = pd.read_json(all_data_url['statisticsData'][x])
    provinceName = all_data_url['provinceName'][x]
    df[f'{provinceName}'] =provinceName
    df = df.loc[:,[f'{provinceName}','data']]
    df_n= df.loc[:,[f'{provinceName}']]
    df_n['dateId']= [x['dateId']
                      for x in df['data']]
    df_n['confirmedCount']= [x['confirmedCount']
                      for x in df['data']]
    df_n['confirmedCount']= [x['confirmedCount']
                      for x in df['data']]
    df_n['curedCount']= [x['curedCount']
                      for x in df['data']]
    df_n['deadCount']= [x['deadCount']
                      for x in df['data']]
    #時間處理程序 
    df_n['dateId']= pd.to_datetime(df_n['dateId'], format='%Y%m%d', errors='ignore')
    to_names.append(df_n)
    pbar.update()

100%|██████████| 31/31 [00:19<00:00,  1.55it/s]


In [7]:
#  use 31count province right join
df_convir= pd.concat(to_names,axis=1,join='inner')
df_convir.head(1)

,内蒙古自治区,dateId,confirmedCount,curedCount,deadCount,吉林省,dateId,confirmedCount,curedCount,deadCount,上海市,dateId,confirmedCount,curedCount,deadCount,湖北省,dateId,confirmedCount,curedCount,deadCount,广东省,dateId,confirmedCount,curedCount,deadCount,北京市,dateId,confirmedCount,curedCount,deadCount,四川省,dateId,confirmedCount,curedCount,deadCount,辽宁省,dateId,confirmedCount,curedCount,deadCount,...,山西省,dateId,confirmedCount,curedCount,deadCount,云南省,dateId,confirmedCount,curedCount,deadCount,贵州省,dateId,confirmedCount,curedCount,deadCount,甘肃省,dateId,confirmedCount,curedCount,deadCount,新疆维吾尔自治区,dateId,confirmedCount,curedCount,deadCount,宁夏回族自治区,dateId,confirmedCount,curedCount,deadCount,青海省,dateId,confirmedCount,curedCount,deadCount,西藏自治区,dateId,confirmedCount,curedCount,deadCount
0,内蒙古自治区,2020-01-23,1,0,0,吉林省,2020-01-23,3,0,0,上海市,2020-01-20,1,0,0,湖北省,2020-01-20,270,25,6,广东省,2020-01-19,1,0,0,北京市,2020-01-20,5,0,0,四川省,2020-01-22,7,0,0,辽宁省,2020-01-22,2,0,0,...,山西省,2020-01-22,1,0,0,云南省,2020-01-24,5,0,0,贵州省,2020-01-23,3,0,0,甘肃省,2020-01-23,2,0,0,新疆维吾尔自治区,2020-01-23,2,0,0,宁夏回族自治区,2020-01-23,2,0,0,青海省,2020-01-25,1,0,0,西藏自治区,2020-01-29,1,0,0


## outter_join
把表格全部連接

In [9]:

data =[]
province_ls= []
with tqdm(total = len(all_data_url) ) as pbar:
  for x in range (len(all_data_url)) :
    df_all = pd.read_json(all_data_url['statisticsData'][x])
    provinceName = all_data_url['provinceName'][x]
    df_all['provincenae'] =provinceName
    df_all = df_all.loc[:,['provincenae','data']]
    province_ls.append(provinceName)
    data.append(df_all)
    pbar.update()

100%|██████████| 31/31 [00:02<00:00, 11.14it/s]


In [11]:
csv_colums = []
province_add = []
with tqdm(total = len(data) ) as pbar:
  for x in data:
    for xs in x['data']:
      
      csv_colums.append({'dateId':xs['dateId'],
          'confirmedCount':xs['confirmedCount'],
          'curedCount':xs['curedCount'],
          'deadCount':xs['deadCount']})
      pbar.update()


with tqdm(total = len(data) ) as pbar:
  for x in data:
    for xs in x['provincenae']:
      
      province_add.append({'provinceName':xs})
      pbar.update()

3836it [00:00, 281937.90it/s]         
3836it [00:00, 400561.41it/s]         


In [0]:
df_csv = pd.DataFrame(csv_colums)
df_name = pd.DataFrame(province_add)
df_allin= pd.concat([df_name,df_csv],axis=1)
#time_Serzise 
df_allin['dateId']= pd.to_datetime(df_allin['dateId'], format='%Y%m%d', errors='ignore')

In [16]:
df_allin.sample(2)

,provinceName,dateId,confirmedCount,curedCount,deadCount
1087,山东省,2020-04-20,787,765,7
285,上海市,2020-02-28,337,279,3


In [0]:
df_allin.to_csv('/content/drive/My Drive/PAPER/df_all.csv')